In [2]:
import torch

e:\download\python\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 框架真正厉害的点？
可以帮我们把反向传播全部计算好

In [3]:
# Method 1 - recommended
x = torch.randn(3,4,requires_grad=True)
x

tensor([[ 0.8091, -1.8436, -1.2494, -0.2183],
        [-0.6858,  0.4545,  0.3285, -0.2726],
        [-0.2179,  1.2607, -0.5618,  0.5449]], requires_grad=True)

In [4]:
# Method 2
x = torch.randn(3,4)
x.requires_grad = True
x

tensor([[-0.2327, -0.3821, -0.9949,  1.0545],
        [-0.2377, -1.7463, -0.9274,  1.5329],
        [ 2.4034,  0.3413, -0.3257, -1.0697]], requires_grad=True)

In [5]:
b = torch.randn(3,4,requires_grad=True)
b

tensor([[-0.2633,  0.1450, -1.7550, -0.4044],
        [-0.7042,  0.8514,  2.0842, -0.7278],
        [ 1.4476,  1.7176, -1.1932,  0.8953]], requires_grad=True)

In [6]:
t = x+b
t

tensor([[-0.4959, -0.2371, -2.7500,  0.6501],
        [-0.9419, -0.8949,  1.1569,  0.8051],
        [ 3.8510,  2.0589, -1.5190, -0.1743]], grad_fn=<AddBackward0>)

In [7]:
y = t.sum()
y

tensor(1.5087, grad_fn=<SumBackward0>)

In [8]:
y.backward()
y

tensor(1.5087, grad_fn=<SumBackward0>)

In [9]:
b.grad
## 这儿的变量就是b，一元一次函数求导

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

虽然没有指定t的required_grad但是需要用到它，也会默认的

In [10]:
x.requires_grad,b.requires_grad,t.requires_grad

(True, True, True)

In [11]:
# calculate flow - example 
x = torch.rand(1)
y = torch.rand(1)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b

In [12]:
# check autograde
x.requires_grad,b.requires_grad,w.requires_grad

(False, True, True)

In [13]:
## again
x = torch.rand(1)
b = torch.rand(1,requires_grad=True)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b

In [14]:
# check autograde
x.requires_grad,b.requires_grad,w.requires_grad,y.requires_grad # y is needed

(False, True, True, True)

In [15]:
x.is_leaf,w.is_leaf,b.is_leaf,y.is_leaf,z.is_leaf
# z = wx+b,是否为叶子结点

(True, True, True, False, False)

In [16]:
# back propagation
z.backward(retain_graph=True) # 注意，如果不清空梯度会累加起来


In [17]:
w.grad

tensor([0.5140])

In [18]:
b.grad

tensor([1.])

线性回归试水

In [19]:
# 构造一组输入数据X与对应的标签Y
import numpy as np
x_values = [i for i in range(11)]
x_train  = np.array(x_values,dtype = np.float32)
x_train = x_train.reshape(-1,1)
x_train.shape

(11, 1)

In [20]:
import numpy as np
y_values = [2*i+1 for i in range(11)]
y_train = np.array(y_values,dtype=np.float32)
y_train = y_train.reshape(-1,1)
y_train.shape

(11, 1)

In [21]:
import torch 
import torch.nn as nn

线性回归模型

In [22]:
# 线性回归就是不添加激活函数的全连接层
class LinearRegressionModel(nn.Module): # 定义了类的名字为LinearRegressionModel,继承了父类nn.Module
    def __init__(self,input_dim,output_dim): # 实例self从父类继承属性input_dim,output_dim
        super(LinearRegressionModel,self).__init__() # super,可以简单地理解为执行父类的__init__函数，
        # 也就是初始化父类，这样才能访问父类的成员
        self.linear = nn.Linear(input_dim,output_dim)
        
    def forward(self,x):
        out = self.linear(x)
        return out

In [23]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim,output_dim,)

In [24]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [25]:
# 制定好参数和损失函数
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion = nn.MSELoss()
# 一般分类任务是交叉熵，回归任务mse，也就是mean square error

In [26]:
# 训练模型
for epoch in range(epochs):
    epoch+=1
    # 注意转为tensor
    inputs = torch.tensor(x_train)
    labels = torch.tensor(y_train)
    
    # 梯度要清零每一次迭代
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model(inputs)
    
    # 计算损失
    loss = criterion(outputs,labels)
    
    # 返回传播
    loss.backward()
    
    # 更新权重
    optimizer.step()
    
    if epoch % 50 == 0:
        print("epoch {},loss {}".format(epoch,loss.item()))
    
    
    

epoch 50,loss 0.006575658451765776
epoch 100,loss 0.003750476287677884
epoch 150,loss 0.0021391164045780897
epoch 200,loss 0.0012200846103951335
epoch 250,loss 0.000695887312758714
epoch 300,loss 0.0003969058452639729
epoch 350,loss 0.00022637330403085798
epoch 400,loss 0.00012911540397908539
epoch 450,loss 7.364393241005018e-05
epoch 500,loss 4.2005183786386624e-05
epoch 550,loss 2.395801857346669e-05
epoch 600,loss 1.3664072866959032e-05
epoch 650,loss 7.794496923452243e-06
epoch 700,loss 4.445927061169641e-06
epoch 750,loss 2.5357885533594526e-06
epoch 800,loss 1.4462606259257882e-06
epoch 850,loss 8.251524832303403e-07
epoch 900,loss 4.705849505626247e-07
epoch 950,loss 2.684837738797796e-07
epoch 1000,loss 1.5291207944301277e-07


In [27]:
# 检测模型预测结果
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[ 1.0007278],
       [ 3.000623 ],
       [ 5.0005183],
       [ 7.0004134],
       [ 9.000309 ],
       [11.000204 ],
       [13.000099 ],
       [14.999994 ],
       [16.99989  ],
       [18.999786 ],
       [20.999681 ]], dtype=float32)

In [28]:
# 模型的保存和读取
torch.save(model.state_dict(),"model.pkl")

In [29]:
model.load_state_dict(torch.load("model.pkl"))

<All keys matched successfully>

# 使用GPU进行训练
只需要将数据和模型传到cuda里面就可以了

In [33]:
# 先检验一下啊gpu可不可以用哈哈
torch.cuda.is_available()


False

In [32]:
import torch
import torch.nn as nn
import numpy as np

class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear = nn.Linear(input_dim,output_dim)
    
    def forward(self,x):
        out = self.linear(x)
        return out
    
input_dim = 1 
output_dim =1 

model = LinearRegressionModel(input_dim,output_dim)

# 搜寻我们的torch的device，主义cuda：0 与cuda 对于单卡玩家是没有区别的
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.MSELoss()

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

epoches = 1000

for epoch in range(epochs):
    epoch+=1
    # 注意转为tensor
    inputs = torch.tensor(x_train)
    labels = torch.tensor(y_train)
    
    # 梯度要清零每一次迭代
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model(inputs)
    
    # 计算损失
    loss = criterion(outputs,labels)
    
    # 返回传播
    loss.backward()
    
    # 更新权重
    optimizer.step()
    
    if epoch % 50 == 0:
        print("epoch {},loss {}".format(epoch,loss.item()))
    



epoch 50,loss 0.2602664828300476
epoch 100,loss 0.1484462320804596
epoch 150,loss 0.08466830849647522
epoch 200,loss 0.04829154163599014
epoch 250,loss 0.027543742209672928
epoch 300,loss 0.015709955245256424
epoch 350,loss 0.008960350416600704
epoch 400,loss 0.005110647529363632
epoch 450,loss 0.002914928598329425
epoch 500,loss 0.0016625456046313047
epoch 550,loss 0.0009482643217779696
epoch 600,loss 0.0005408604047261178
epoch 650,loss 0.0003084798518102616
epoch 700,loss 0.00017594677046872675
epoch 750,loss 0.00010034978186013177
epoch 800,loss 5.7235025451518595e-05
epoch 850,loss 3.264545739511959e-05
epoch 900,loss 1.8619555703480728e-05
epoch 950,loss 1.061913735611597e-05
epoch 1000,loss 6.057673090253957e-06
